# Project 1, Part 4, Holiday Related Query



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.4 Find holiday days, and days within a one week of a holiday, where the actual sales differ by more than 15% from expected sales

For each day of week, find the average dollar amount of sales for that day of week. We will call this the expected sales.

For each holiday in the holidays table consider days +/- 7 days from that date. We will call this the date considered.  

For each date considered, note the day of week.

For each date considered, sum the total dollar amount of sales for that date.  We will call this the actual sales.

For each date considered, find the ratio of actual sales (for the date considered) to expected sales (for that day of week).

Return dates where the ratio is less than 0.85

Note that data before January 1, 2020 is not present in the database, so of course, will not be analyzed since it does not exist.

Note that days where the store was closed should also be included, with 0 for the actual sales.

Sort by date analyzed.

You may find the following subquery helpful:
```
     select my_date::date
     from generate_series('2020-01-01', '2020-12-31', '1 day'::interval) my_date
```

Write 1 and only 1 query.  Note that the query may have as many subqueries, including with clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Sort data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return 26 rows into a Pandas dataframe and should look similar to this  (note that markdown drops the trailing zero while Pandas does not - this is fine): 

||holiday_name|date_analyzed|day_of_week|actual_sales_dollars|expected_sales_dollars|ratio_actual_expected|
|---|---|---|---|---|---|---|
|0|New Year's Day|2020-01-01|Wednesday|133776|263256|0.51|
|1|MLK Day|2020-01-17|Friday   |127092|252522|0.5|
|2|MLK Day|2020-01-18|Saturday |135204|373490|0.36|
|3|MLK Day|2020-01-19|Sunday   |130368|357482|0.36|
|4|MLK Day|2020-01-20|Monday   |130740|253225|0.52|
|5|President's Day|2020-02-14|Friday   |133452|252522|0.53|
|6|President's Day|2020-02-15|Saturday |132096|373490|0.35|
|7|President's Day|2020-02-16|Sunday   |132180|357482|0.37|
|8|President's Day|2020-02-17|Monday   |135228|253225|0.53|
|9|Easter|2020-04-12|Sunday   |136164|357482|0.38|
|10|Mother's Day|2020-05-10|Sunday   |134496|357482|0.38|
|11|Memorial Day|2020-05-22|Friday   |129132|252522|0.51|
|12|Memorial Day|2020-05-23|Saturday |132900|373490|0.36|
|13|Memorial Day|2020-05-24|Sunday   |130764|357482|0.37|
|14|Memorial Day|2020-05-25|Monday   |132624|253225|0.52|
|15|Father's Day|2020-06-21|Sunday   |135120|357482|0.38|
|16|Independence Day|2020-07-03|Friday   |132960|252522|0.53|
|17|Independence Day|2020-07-04|Saturday |134952|373490|0.36|
|18|Independence Day|2020-07-05|Sunday   |133884|357482|0.37|
|19|Independence Day|2020-07-06|Monday   |134964|253225|0.53|
|20|Labor Day|2020-09-04|Friday   |132048|252522|0.52|
|21|Labor Day|2020-09-05|Saturday |132840|373490|0.36|
|22|Labor Day|2020-09-06|Sunday   |133332|357482|0.37|
|23|Labor Day|2020-09-07|Monday   |130020|253225|0.51|
|24|Thanksgiving|2020-11-26|Thursday |0|266051|0.0|
|25|Christmas|2020-12-25|Friday   |0|252522|0.0|



In [4]:
rollback_before_flag = True
rollback_after_flag = True

#reference table for expected_sales
query = """


with actual_sales as ((select sale_date, sum(total_amount) as actual_sales_dollars from sales group by sale_date)
 union 
(select holiday_date, 0 from holidays)
order by sale_date),

expected_sales as (select day_of_week, round(avg(total_sales)) as expected_sales_dollars
                from (select to_char(sales.sale_date, 'Day') as day_of_week,
                     sum(total_amount) as total_sales
                     from sales
                    group by sale_date) as a
                    group by day_of_week),
        
dates as (select description as holiday_name, cast(my_date as DATE) as date_analyzed, to_char(my_date, 'Day') as day_of_week
        FROM generate_series('2020-01-01', '2020-12-31', '1 day'::interval) my_date
        JOIN holidays 
            ON my_date BETWEEN holiday_date - interval '7 days' AND holiday_date + interval '7 days')

select dates.*, 
    sum(actual_sales.actual_sales_dollars) as actual_sales_dollars,
    avg(expected_sales.expected_sales_dollars) as expected_sales_dollars,
    round(sum(actual_sales.actual_sales_dollars)/avg(expected_sales.expected_sales_dollars),2) as ratio_actual_expected
    from dates
        join actual_sales
            on dates.date_analyzed = actual_sales.sale_date
        join expected_sales
            on dates.day_of_week = expected_sales.day_of_week
    group by date_analyzed, dates.day_of_week, holiday_name
    having sum(actual_sales.actual_sales_dollars)/avg(expected_sales.expected_sales_dollars) < 0.85
    
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,holiday_name,date_analyzed,day_of_week,actual_sales_dollars,expected_sales_dollars,ratio_actual_expected
0,New Year's Day,2020-01-01,Wednesday,133776,263256,0.51
1,MLK Day,2020-01-17,Friday,127092,252522,0.50
2,MLK Day,2020-01-18,Saturday,135204,373490,0.36
3,MLK Day,2020-01-19,Sunday,130368,357482,0.36
4,MLK Day,2020-01-20,Monday,130740,253225,0.52
5,President's Day,2020-02-14,Friday,133452,252522,0.53
6,President's Day,2020-02-15,Saturday,132096,373490,0.35
7,President's Day,2020-02-16,Sunday,132180,357482,0.37
8,President's Day,2020-02-17,Monday,135228,253225,0.53
9,Easter,2020-04-12,Sunday,136164,357482,0.38
